# Ergebnisse der Zwei-Stufen-XGBoost-Experimente

In diesem Notebook werte ich die gespeicherten Ergebnis-Dateien aus:

- `notebooks/results/two_stage__<EXP_ID>.json`

Jedes JSON gehört zu einem bestimmten Experiment (`EXP_ID`) und enthält:

- Metriken für das Signal-Modell (Train/Val/Test),
- Metriken für das Richtungs-Modell (Train/Val/Test),
- kombinierte 3-Klassen-Auswertung (neutral / up / down) auf dem Test-Split.

Ziel ist es, verschiedene Experimente (z.B. unterschiedliche Label-Parameter
oder Oversampling-Faktoren) **nebeneinander vergleichen** zu können.


In [ ]:
import sys
from pathlib import Path
import json

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")

# Projektwurzel finden, indem wir nach oben laufen, bis ein Ordner `src` existiert.
cwd = Path.cwd()
project_root = cwd
while not (project_root / "src").is_dir():
    project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

results_dir = project_root / "notebooks" / "results"
print("Results-Ordner:", results_dir)

# Alle Ergebnisdateien einlesen, die dem Schema two_stage__*.json entsprechen.
json_paths = sorted(results_dir.glob("two_stage__*.json"))
print("Gefundene Experimente:")
for p in json_paths:
    print(" -", p.name)


## 1. Übersichtstabelle für Signal- und Richtungs-Modell (Test)

In dieser Zelle sammle ich aus jeder JSON-Datei die wichtigsten Kennzahlen
für den **Test-Split**:

- Signal-Modell (`signal`): Accuracy, Precision, Recall, F1 der Klasse `move`.
- Richtungs-Modell (`direction`): Accuracy, Precision, Recall, F1 der Klasse `up`.

Das Ergebnis ist eine Tabelle, in der jede Zeile ein Experiment (`exp_id`) beschreibt.


In [ ]:
rows = []

for path in json_paths:
    with open(path, "r") as f:
        data = json.load(f)

    exp_id = path.stem.replace("two_stage__", "")

    # Signal-Modell (Test)
    sig_test = data["signal"]["test"]
    sig_report = sig_test["report"]
    sig_move = sig_report.get("move", {})  # Klasse 'move'

    # Richtungs-Modell (Test)
    dir_test = data["direction"]["test"]
    dir_report = dir_test["report"]
    dir_up = dir_report.get("up", {})  # Klasse 'up'

    rows.append(
        {
            "exp_id": exp_id,
            # Signal (move)
            "signal_acc": sig_report.get("accuracy", float("nan")),
            "signal_precision_move": sig_move.get("precision", float("nan")),
            "signal_recall_move": sig_move.get("recall", float("nan")),
            "signal_f1_move": sig_move.get("f1-score", float("nan")),
            # Direction (up)
            "direction_acc": dir_report.get("accuracy", float("nan")),
            "direction_precision_up": dir_up.get("precision", float("nan")),
            "direction_recall_up": dir_up.get("recall", float("nan")),
            "direction_f1_up": dir_up.get("f1-score", float("nan")),
        }
    )

summary_df = pd.DataFrame(rows)
summary_df


## 2. Experimente visuell vergleichen

Als nächstes stelle ich Precision und Recall der Klasse `move` (Signal-Modell)
und der Klasse `up` (Richtungs-Modell) für alle Experimente als Balkendiagramme dar.


In [ ]:
if not summary_df.empty:
    # Sortierung z.B. nach Precision der 'move'-Klasse
    summary_df_sorted = summary_df.sort_values("signal_precision_move", ascending=False)

    plt.figure(figsize=(10, 4))
    sns.barplot(data=summary_df_sorted, x="exp_id", y="signal_precision_move")
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Precision (signal=move)")
    plt.title("Signal-Modell: Precision für 'move' pro Experiment")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 4))
    sns.barplot(data=summary_df_sorted, x="exp_id", y="signal_recall_move")
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Recall (signal=move)")
    plt.title("Signal-Modell: Recall für 'move' pro Experiment")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 4))
    sns.barplot(data=summary_df_sorted, x="exp_id", y="direction_precision_up")
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Precision (direction=up)")
    plt.title("Richtungs-Modell: Precision für 'up' pro Experiment")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 4))
    sns.barplot(data=summary_df_sorted, x="exp_id", y="direction_recall_up")
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Recall (direction=up)")
    plt.title("Richtungs-Modell: Recall für 'up' pro Experiment")
    plt.tight_layout()
    plt.show()
else:
    print("Keine Ergebnis-Dateien gefunden. Bitte zuerst Experimente ausführen und JSONs speichern.")
